In [2]:
options(warn=-1)
memory.size() ### Checking your memory size
memory.limit() ## Checking the set limit
memory.limit(size=560000) ### expanding your memory 
memory.limit()

[1] 45.53

[1] 8066

[1] 560000

[1] 560000

In [3]:
#esqueleto de grid search
#se espera que los alumnos completen lo que falta para recorrer TODOS cuatro los hiperparametros 

rm( list=ls() )  #Borro todos los objetos
gc()   #Garbage Collection

require("data.table")
require("rpart")
require("parallel")

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,480071,25.7,1060675,56.7,621911,33.3
Vcells,877469,6.7,8388608,64.0,1601322,12.3


Loading required package: data.table
Loading required package: rpart
Loading required package: parallel


In [4]:
options(repr.plot.width = 20, repr.plot.height = 10) 
setwd("C:\\Users\\oliva\\OneDrive\\Desktop\\DS\\Austral\\08 - Labo1") #Aqui se debe poner la ruta de la PC local

In [5]:
ksemillas <- c(477293, 525713, 679561, 895627, 978643) #reemplazar por las propias semillas
semilla <- c(477293)

In [6]:
#particionar agrega una columna llamada fold a un dataset que consiste en una particion estratificada segun agrupa
# particionar( data=dataset, division=c(70,30), agrupa=clase_ternaria, seed=semilla)   crea una particion 70, 30 

particionar <- function(data, division, agrupa = " ",  campo = "fold", start = 1,  seed = NA)
{
  if(!is.na(seed)){  #Si la semilla no esta vacia
      set.seed(seed) #Settearla con seed (el parametro enviado a la funcion)
                  }
    
  bloque <- unlist(mapply(function(x,y) {rep(y, x)}, division, seq(from = start, length.out = length(division))))  
    
  data[, (campo) :=  sample(rep(bloque, ceiling(.N/length(bloque))))[1:.N],
          by = agrupa] #todas las filas, crear (o asignar, es flexible depende si existe o no) una columna campo con ese contenido, agrupando por agrupa (en este caso clase ternaria)
}


In [7]:
ArbolEstimarGanancia  <- function( semilla, param_basicos )
{
  #particiono estratificadamente el dataset
  particionar(dataset, division = c (70, 30), agrupa = "clase_ternaria", seed = semilla)  #Cambiar por la primer semilla de cada uno !

  #genero el modelo
  modelo  <- rpart("clase_ternaria ~ .",     #quiero predecir clase_ternaria a partir del resto
                   data = dataset[fold == 1],  #fold==1  es training,  el 70% de los datos
                   xval = 0,
                   control = param_basicos )  #aqui van los parametros del arbol

  #aplico el modelo a los datos de testing
  prediccion  <- predict(modelo,   #el modelo que genere recien
                          dataset[fold==2],  #fold==2  es testing, el 30% de los datos
                          type= "prob") #type= "prob"  es que devuelva la probabilidad

  #prediccion es una matriz con TRES columnas, llamadas "BAJA+1", "BAJA+2"  y "CONTINUA"
  #cada columna es el vector de probabilidades 

  #calculo la ganancia en testing  que es fold==2
  ganancia_test  <- dataset[ fold==2, 
                             sum(ifelse( prediccion[, "BAJA+2"]  >  1/60,
                                         ifelse(clase_ternaria=="BAJA+2", 59000, -1000 ),
                                         0 ) )]

  #escalo la ganancia como si fuera todo el dataset
  ganancia_test_normalizada <- ganancia_test/0.3

  return(ganancia_test_normalizada)
}

In [8]:
ArbolesMontecarlo  <- function(semillas, param_basicos )
{
  #la funcion mcmapply  llama a la funcion ArbolEstimarGanancia  tantas veces como valores tenga el vector  ksemillas
  ganancias  <- mcmapply( ArbolEstimarGanancia, 
                          semillas,   #paso el vector de semillas, que debe ser el primer parametro de la funcion ArbolEstimarGanancia
                          MoreArgs= list( param_basicos),  #aqui paso el segundo parametro
                          SIMPLIFY= FALSE,
                          mc.cores= 1 )  #se puede subir a 5 si posee Linux o Mac OS

  ganancia_promedio  <- mean( unlist(ganancias) )

  return( ganancia_promedio )
}


In [9]:
#Aqui se debe poner la carpeta de la computadora local
setwd("C:\\Users\\oliva\\OneDrive\\Desktop\\DS\\Austral\\08 - Labo1\\labo")  #Establezco el Working Directory

#cargo los datos
dataset  <- fread("./datasets/paquete_premium_202011.csv")


#genero el archivo para Kaggle
#creo la carpeta donde va el experimento
# HT  representa  Hiperparameter Tuning
#dir.create( "./labo/exp/",  showWarnings = FALSE ) 
dir.create( "./exp/HT2021/", showWarnings = FALSE )
archivo_salida  <- "./exp/HT2021/gridsearch7.txt"


In [10]:
#Escribo los titulos al archivo donde van a quedar los resultados
#atencion que si ya existe el archivo, esta instruccion LO SOBREESCRIBE, y lo que estaba antes se pierde
#la forma que no suceda lo anterior es con append=TRUE
cat( file=archivo_salida,
     sep= "",
     "cp", "\t",
     "max_depth", "\t",
     "min_split", "\t",
     "minbucket", "\t",
     "ganancia_promedio", "\n")

In [11]:
#itero por los loops anidados para cada hiperparametro
for( cp  in  c(-0.799)  )
{
for( vmax_depth  in  c(6)  )
{
for( vmin_split  in  c(1310, 1320, 1330, 1340, 1345, 1355, 1362, 1370, 1380, 1390)  )
{
for( vmin_bucket  in  c(610, 620, 630, 640, 650, 660, 678)  )
{ 
  #cp = -0.1
  #notar como se agrega
  param_basicos  <- list( "cp"=         cp,       #complejidad minima
                          "minsplit"=  vmin_split,  #minima cantidad de registros en un nodo para hacer el split
                          "minbucket"=  vmin_bucket,          #minima cantidad de registros en una hoja
                          "maxdepth"=  vmax_depth ) #profundidad máxima del arbol

  #Un solo llamado, con la semilla 17
  ganancia_promedio  <- ArbolesMontecarlo( ksemillas,  param_basicos )

 #escribo los resultados al archivo de salida
  cat(  file=archivo_salida,
        append= TRUE,
        sep= "",
        cp, "\t", 
        vmax_depth, "\t",
        vmin_split, "\t",
        vmin_bucket, "\t",  
        ganancia_promedio, "\n"  )

}
}
}
}

In [ ]:
#itero por los loops anidados para cada hiperparametro

for( vmax_depth  in  c( 4, 6 )  )
{
for( vmin_split  in  c(600,500, 400, 200 )  ) 
{

  #notar como se agrega
  param_basicos  <- list( "cp"=         -0,1,       #complejidad minima
                          "minsplit"=  vmin_split,  #minima cantidad de registros en un nodo para hacer el split
                          "minbucket"=  4,          #minima cantidad de registros en una hoja
                          "maxdepth"=  vmax_depth ) #profundidad máxima del arbol

  #Un solo llamado, con la semilla 17
  ganancia_promedio  <- ArbolesMontecarlo( ksemillas,  param_basicos )

  #escribo los resultados al archivo de salida
  cat(  file=archivo_salida,
        append= TRUE,
        sep= "",
        vmax_depth, "\t",
        vmin_split, "\t",
        ganancia_promedio, "\n"  )

}
}